In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn import metrics

# from mlxtend.plotting import plot_decision_regions
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from ast import literal_eval

import warnings
import numpy as np
from collections import OrderedDict

from lob_data_utils import lob, db_result, model, roc_results
from lob_data_utils.svm_calculation import lob_svm
import os


sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

In [2]:
data_length = 10000
r = 0.1
s = 1.0
stocks = list(roc_results.result_cv_10000.keys())

In [3]:
def get_mean_scores(scores: dict) -> dict:
    mean_scores = {}
    for k, v in scores.items():
        mean_scores[k] = np.mean(v)
    return mean_scores

def get_score_for_clf(clf, df_test):
    x_test = df_test[['queue_imbalance']]
    y_test = df_test['mid_price_indicator'].values
    return model.test_model(clf, x_test, y_test)

def get_logistic_regression(stock, data_length):
    df, df_test = lob.load_prepared_data(
        stock, data_dir='../gaussian_filter/data', cv=False, length=data_length)
    clf = LogisticRegression()
    train_x = df[['queue_imbalance']]

    scores = model.validate_model(clf, train_x, df['mid_price_indicator'])
    res = {
        **get_mean_scores(scores),
        'stock': stock,
        'kernel': 'logistic',
    }
    test_scores = get_score_for_clf(clf, df_test)
    return {**res, **test_scores}

In [4]:
df_res = pd.DataFrame()
for stock in stocks:
    #pd.read_csv('svm_features_{}_len{}_r{}_s{}.csv'.format(stock, data_length, r, s))
    filename = 'svm_pca_gdf_{}_len{}_r{}_s{}.csv'.format(stock, data_length, r, s)
    if os.path.exists(filename):
        df_res = df_res.append(pd.read_csv(filename))
#df_res.drop(columns=['Unnamed: 0'], inplace=True)
columns = ['C', 'f1', 'features', 'gamma', 'kappa',
           'matthews', 'roc_auc', 'stock',
       'test_f1', 'test_kappa', 'test_matthews', 'test_roc_auc']
df_res[columns].sort_values(by='matthews', ascending=False).groupby('stock').head(1)

,C,f1,features,gamma,kappa,matthews,roc_auc,stock,test_f1,test_kappa,test_matthews,test_roc_auc
35,100.0,0.601556,gdf_0-50_que,0.001,0.190221,0.191987,0.594991,11946,0.625294,0.199763,0.199985,0.599691
16,0.1,0.594436,pca_gdf_que4,0.100,0.182123,0.183503,0.591330,3879,0.575435,0.122216,0.122692,0.561159
37,100.0,0.590380,gdf_24-26_que_prev,0.100,0.168715,0.170007,0.584327,3035,0.573872,0.100655,0.100797,0.550257
42,1000.0,0.563819,pca_gdf_que_prev10,0.001,0.166019,0.167706,0.583163,7858,0.516934,0.176826,0.181780,0.587530
31,10.0,0.626963,gdf_24-26_que_prev,1.000,0.164714,0.166758,0.582027,4320,0.630831,0.160028,0.161926,0.579402
37,100.0,0.590188,pca_gdf_que5,0.100,0.157928,0.163450,0.579142,1956,0.588512,0.125956,0.127068,0.563007
42,1000.0,0.580410,pca_gdf_que_prev9,0.001,0.156176,0.158632,0.578438,9761,0.610460,0.133328,0.135927,0.566502
28,10.0,0.571921,gdf_0-50_que_prev,0.001,0.152830,0.158056,0.576684,12417,0.593237,0.158458,0.159192,0.579324
35,100.0,0.613076,gdf_0-50_que_prev,0.001,0.156064,0.157804,0.577747,10484,0.567397,0.112125,0.112700,0.556190
42,1000.0,0.588744,gdf_0-50_que_prev,0.001,0.153905,0.157275,0.577143,1472,0.634786,0.118508,0.121767,0.558322


In [5]:
log_res = []
for stock in stocks:
    log_res.append(get_logistic_regression(stock, data_length))
df_log_res = pd.DataFrame(log_res)
df_log_res['stock'] = df_log_res['stock'].values.astype(np.int)
df_log_res.index = df_log_res['stock'].values.astype(np.int)

In [6]:
df_gdf_best = df_res[columns].sort_values(by='test_matthews', ascending=False).groupby('stock').head(1)
df_gdf_best['stock'] = df_gdf_best['stock'].values.astype(np.int)
df_gdf_best.index = df_gdf_best['stock'].values.astype(np.int)

In [7]:
df_all = pd.merge(df_gdf_best, df_log_res, on='stock', suffixes=['_svm', '_log'])

In [13]:
all_columns = [ 'features', 'matthews_svm', 'matthews_log',  'test_matthews_svm',  'test_matthews_log',
       'roc_auc_svm', 'roc_auc_log', 'test_roc_auc_svm',  'test_roc_auc_log', 'stock', 
               'f1_svm', 'f1_log', 'test_f1_svm', 'test_f1_log', 'stock' ]
df_all[all_columns]

,features,matthews_svm,matthews_log,test_matthews_svm,test_matthews_log,roc_auc_svm,roc_auc_log,test_roc_auc_svm,test_roc_auc_log,stock,f1_svm,f1_log,test_f1_svm,test_f1_log,stock
0,gdf_0-50_que,0.186447,0.186824,0.209920,0.203627,0.592012,0.592373,0.603395,0.601087,11946,0.604273,0.595737,0.645307,0.634056,11946
1,pca_gdf_que_prev10,0.167706,0.146384,0.181780,0.171235,0.583163,0.572852,0.587530,0.584512,7858,0.563819,0.551977,0.516934,0.539665,7858
2,gdf_24-26_que_prev,0.159758,0.156657,0.175396,0.163789,0.574381,0.577752,0.586201,0.581178,4320,0.572707,0.610483,0.634821,0.620721,4320
3,pca_gdf_que1,0.142380,0.137272,0.171827,0.168692,0.571041,0.567489,0.585445,0.583011,3161,0.583146,0.579932,0.610090,0.621412,3161
4,gdf_0-50_que_prev,0.143412,0.150172,0.171794,0.154446,0.568192,0.573432,0.584803,0.576707,12417,0.583898,0.591476,0.612302,0.596958,12417
5,pca_gdf_que_prev7,0.134408,0.130301,0.170011,0.146515,0.562815,0.564461,0.583777,0.573201,1113,0.475626,0.534083,0.549462,0.591563,1113
6,pca_gdf_que1,0.129062,0.125634,0.168666,0.162023,0.564221,0.562340,0.584213,0.580890,2651,0.561284,0.553881,0.600577,0.597790,2651
7,pca_gdf_que5,0.075843,0.142499,0.166840,0.162155,0.535618,0.570582,0.580780,0.580693,10508,0.630194,0.582458,0.620845,0.595089,10508
8,pca_gdf_que10,0.128650,0.131577,0.165576,0.150530,0.563957,0.564686,0.579491,0.570781,2602,0.591001,0.612748,0.638925,0.641621,2602
9,que,0.047276,0.138016,0.162363,0.138375,0.521266,0.568685,0.578129,0.568627,1431,0.656961,0.585431,0.627866,0.593882,1431


In [9]:
len(df_all[df_all['matthews_svm'] > df_all['matthews_log']][all_columns]), len(df_all)

(18, 53)

In [10]:
len(df_all[df_all['roc_auc_svm'] > df_all['roc_auc_log']][all_columns]), len(df_all)

(15, 53)

In [14]:
df_all[df_all['test_matthews_svm'] < df_all['test_matthews_log']][all_columns]

,features,matthews_svm,matthews_log,test_matthews_svm,test_matthews_log,roc_auc_svm,roc_auc_log,test_roc_auc_svm,test_roc_auc_log,stock,f1_svm,f1_log,test_f1_svm,test_f1_log,stock
15,gdf_0-50_que_prev,0.143689,0.145601,0.149897,0.153016,0.571379,0.572127,0.574918,0.576301,11869,0.576742,0.582713,0.570273,0.561531,11869
29,pca_gdf_que_prev2,0.124144,0.130253,0.125377,0.134860,0.559962,0.564948,0.562432,0.567422,9086,0.523951,0.554918,0.531972,0.567149,9086
42,gdf_20_30_que,0.112966,0.113968,0.103896,0.106202,0.556379,0.556876,0.551929,0.553061,9269,0.546165,0.546408,0.564477,0.567554,9269
49,pca_gdf_que6,0.075776,0.132624,0.085385,0.086034,0.534466,0.563571,0.537391,0.538530,13003,0.273337,0.479875,0.392788,0.409548,13003
50,gdf_24-26_que_prev,0.113738,0.111922,0.076066,0.086004,0.555466,0.554976,0.537993,0.543004,9063,0.514924,0.525106,0.529052,0.545274,9063


In [12]:
df_all[df_all['test_roc_auc_svm'] < df_all['test_roc_auc_log']][all_columns]

,features,matthews_svm,matthews_log,test_matthews_svm,test_matthews_log,roc_auc_svm,roc_auc_log,test_roc_auc_svm,test_roc_auc_log,stock,f1_svm,f1_log,test_f1_svm,test_f1_log
15,gdf_0-50_que_prev,0.143689,0.145601,0.149897,0.153016,0.571379,0.572127,0.574918,0.576301,11869,0.576742,0.582713,0.570273,0.561531
29,pca_gdf_que_prev2,0.124144,0.130253,0.125377,0.134860,0.559962,0.564948,0.562432,0.567422,9086,0.523951,0.554918,0.531972,0.567149
42,gdf_20_30_que,0.112966,0.113968,0.103896,0.106202,0.556379,0.556876,0.551929,0.553061,9269,0.546165,0.546408,0.564477,0.567554
43,que,0.105879,0.109434,0.103763,0.103686,0.546818,0.549733,0.550479,0.550556,11867,0.388810,0.412946,0.471014,0.473273
44,gdf_0-50_que,0.130437,0.136666,0.101526,0.096997,0.562312,0.567394,0.547687,0.547942,4060,0.621179,0.592910,0.605956,0.573308
49,pca_gdf_que6,0.075776,0.132624,0.085385,0.086034,0.534466,0.563571,0.537391,0.538530,13003,0.273337,0.479875,0.392788,0.409548
50,gdf_24-26_que_prev,0.113738,0.111922,0.076066,0.086004,0.555466,0.554976,0.537993,0.543004,9063,0.514924,0.525106,0.529052,0.545274
51,gdf_20_30_que_prev,0.015351,0.117283,0.075347,0.074043,0.505328,0.557431,0.532888,0.537000,9058,0.223283,0.539410,0.624296,0.528993
